<a href="https://colab.research.google.com/github/vigneshwaran-dev/CV-research-timeline/blob/main/ResNet/resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, Input, Add, ZeroPadding2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD

Defining the Model as per the Original Paper

In [3]:
def residual_block(model, filter_map):
  residue = model

  # 1st Layer
  model = Conv2D(filters=filter_map[0], kernel_size=(1, 1), strides=(1, 1), padding='valid')(model)
  model = BatchNormalization()(model)
  model = Activation('relu')(model)

  # 2nd Layer
  model = Conv2D(filters=filter_map[1], kernel_size=(3, 3), strides=(1, 1), padding='same')(model)
  model = BatchNormalization()(model)
  model = Activation('relu')(model)

  # 3rd Layer
  model = Conv2D(filters=filter_map[2], kernel_size=(1, 1), strides=(1, 1), padding='valid')(model)
  model = BatchNormalization()(model)
  model = Add()([model, residue])
  model = Activation('relu')(model)

  return model

def residual_skip(model, filter_map, stride):
  residue = Conv2D(filters=filter_map[2], kernel_size=(1, 1), strides=(stride, stride), padding='valid')(model)
  residue = BatchNormalization()(residue)
  
  # 1st Layer
  model = Conv2D(filters=filter_map[0], kernel_size=(1, 1), strides=(stride, stride), padding='valid')(model)
  model = BatchNormalization()(model)
  model = Activation('relu')(model)

  # 2nd Layer
  model = Conv2D(filters=filter_map[1], kernel_size=(3, 3), strides=(1, 1), padding='same')(model)
  model = BatchNormalization()(model)
  model = Activation('relu')(model)

  # 3rd Layer
  model = Conv2D(filters=filter_map[2], kernel_size=(3, 3), strides=(1, 1), padding='same')(model)
  model = BatchNormalization()(model)
  model = Add()([model, residue])
  model = Activation('relu')(model)

  return model

_input = Input(shape=(224, 224, 3))

# 1st Block
model = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2))(_input)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(model)

# 2nd Block
model = residual_skip(model, [64, 256, 256], 1)
model = residual_block(model, [64, 256, 256])
model = residual_block(model, [64, 256, 256])

# 3rd Block
model = residual_skip(model, [128, 512, 512], 2)
model = residual_block(model, [128, 512, 512])
model = residual_block(model, [128, 512, 512])
model = residual_block(model, [128, 512, 512])

# 4th Block
model = residual_skip(model, [256, 1024, 1024], 2)
model = residual_block(model, [256, 1024, 1024])
model = residual_block(model, [256, 1024, 1024])
model = residual_block(model, [256, 1024, 1024])
model = residual_block(model, [256, 1024, 1024])
model = residual_block(model, [256, 1024, 1024])

# 5th Block
model = residual_skip(model, [512, 2048, 2048], 2)
model = residual_block(model, [512, 2048, 2048])
model = residual_block(model, [512, 2048, 2048])

model = AveragePooling2D(pool_size=(2, 2), padding='same')(model)
model = Flatten()(model)
model = Dense(1000)(model)
model = Activation('softmax')(model)

model = Model(inputs=_input, outputs=model, name='ResNet50')

In [4]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 109, 109, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 109, 109, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 109, 109, 64) 0           batch_normalization[0][0]        
___________________________________________________________________________________________

In [5]:
model.compile(loss=categorical_crossentropy,
                 optimizer=SGD(learning_rate=0.01),
                 metrics=['accuracy'])

Considering the data to be present in TRAIN_DATA_LOCATION and VALIDATION_DATA_LOCATION directories and running them through data generators to perform live data augumentation during the training process

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'TRAIN_DATA_LOCATION'
valid_dir = 'VALIDATION_DATA_LOCATION'

BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=BATCH_SIZE,
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode='categorical')

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=BATCH_SIZE,
                                                    seed=7,
                                                    shuffle=True,
                                                    class_mode='categorical')

train_num = train_generator.samples

Training the Model

In [ ]:
import datetime

log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callback_list = [tensorboard_callback]

model.fit(train_generator,
          epochs=1,
          steps_per_epoch=train_num // BATCH_SIZE,
          validation_data=valid_generator,
          validation_steps=valid_num // BATCH_SIZE,
          callbacks=callback_list,
          verbose=1)

model.save('vgg19.h5')

Visualizing the performance using Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

Prediction

In [ ]:
x_valid, label_batch  = next(iter(valid_generator))
prediction_values = model.predict_classes(x_valid)
print(prediction_values)